In [10]:
import glob
import time
import timeit
import pickle
import requests
import json
import re
import numpy as np
import pandas as pd
from multiprocessing import Pool
from matplotlib import pyplot as plt
from zipfile import ZipFile
import lxml.etree as etree
import xmlschema
from pprint import pprint

# If 'get_career' is a custom module you've created, keep this import
from get_career import *

# Define 'toc' function once
def toc(start_time):
    elapsed = timeit.default_timer() - start_time
    print(elapsed)

# Record start time
start_time = timeit.default_timer()

import math
import matplotlib as mpl
import seaborn as sns
import matplotlib.cm as cm
import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.transforms as mtransforms
import pandas as pd
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
sns.set_context('talk')#, font_scale=1.5)
mpl.rcParams.update({'text.usetex': False})
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
path_career='/mnt/sdb1/sandeep/openalex_ACTIV/'
# GB_INLCUDED_OR_NOT='GB_NOT_INCLUDED'
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec

search in browser:
    https://api.openalex.org/institutions?search=IISER

# FUNCTIONS

In [37]:
def extract_paper_details(papers):
    pubyears = []
    institutions = []
    countries = []
    authors=[]
    for paper in papers:
        # Extract publication year
        pubyears.append(paper.get('publication_year', None))

        # Extract country codes and institutions
        paper_countries = []
        paper_institutions = []
        paper_authors=[]
        
        for authorship in paper.get('authorships', []):

            for inst in authorship.get('institutions', []):
                if 'country_code' in inst:
                    paper_countries.append(inst['country_code'])
                if 'display_name' in inst:
                    paper_institutions.append(inst['display_name'])
#         authors.append(paper_authors)
        countries.append(paper_countries)
        institutions.append(paper_institutions)

    return pubyears, institutions, countries


# pubyears, institutions, countries = extract_paper_details(papers)

# Now you have three lists: pubyears, institutions, and countries


# IISERs 

https://api.openalex.org/institutions?search=indian%20institute%20of%20science%20education%20research

id: "https://openalex.org/I181996519",
id: "https://openalex.org/I127439422",
id: "https://openalex.org/I165916956",
id: "https://openalex.org/I288749910",
id: "https://openalex.org/I4210140410",
id: "https://openalex.org/I50557253",
id: "https://openalex.org/I4210115612",


In [24]:
url

'https://api.openalex.org/works?filter=institutions.id:I181996519'

In [13]:
insti_id="I181996519"

url='https://api.openalex.org/works?filter=institutions.id:'+insti_id
headers = {
        'User-Agent': 'chowdhary_sandeep',
        'From': 'chowdhary_sandeep@phd.ceu.edu'  # This is another valid field
    }

        
cursor='*'
works_=[];it=0
response = requests.get(url+'&per-page=200&cursor='+cursor,headers=headers)
res = json.loads(response.content.decode("utf-8"))
works_.append(res)
totalpages=math.ceil(res['meta']['count']/200)

while not(res['meta']['next_cursor'] is None):
    it+=1
    print(it/totalpages,end='\r')
    cursor=res['meta']['next_cursor']
    try:
        response = requests.get(url+'&per-page=200&cursor='+cursor,headers=headers)
        response.raise_for_status()  # raises exception when not a 2xx response

    except:
        response = requests.get(url+'&per-page=50&cursor='+cursor,headers=headers)
        response.raise_for_status()  # raises exception when not a 2xx response
    if response.status_code != 204:
#                             res= response.json()                            
        res = json.loads(response.content.decode("utf-8"))

        if len(res['results'])>0:
            works_.append(res)
papers=[works['results'] for works in works_]
papers=np.concatenate(papers)
with open(path_career+'works_'+insti_id+'.pkl', 'wb') as f:
    pickle.dump(papers, f) 


In [27]:
# works_IISER[0]



In [15]:
with open(path_career+'works_'+insti_id+'.pkl', 'rb') as f:
    works_IISER=pickle.load(f) 


In [16]:
len(works_IISER)

5199

In [38]:
pubyears, institutions, countries = extract_paper_details(works_IISER)


[['US',
  'NL',
  'US',
  'US',
  'IN',
  'US',
  'NL',
  'US',
  'US',
  'IN',
  'US',
  'NL',
  'US',
  'US',
  'IN'],
 ['US', 'IN', 'US', 'US', 'US', 'US', 'US', 'US', 'US', 'US', 'US'],
 ['IN', 'IN', 'IN', 'IN', 'IN'],
 ['IN', 'IN', 'IN', 'IN', 'IN', 'IN'],
 ['DE',
  'GB',
  'IN',
  'BE',
  'US',
  'BE',
  'IN',
  'CN',
  'SA',
  'ES',
  'CN',
  'US',
  'US',
  'SG',
  'US',
  'US',
  'CN',
  'GB',
  'DE',
  'CN',
  'DE',
  'CH',
  'BE',
  'SE',
  'JP',
  'US',
  'CN',
  'CN',
  'CN',
  'CN',
  'US',
  'IN',
  'US',
  'DE',
  'CN',
  'IN',
  'IN',
  'US',
  'CH',
  'US',
  'US',
  'CN',
  'CN',
  'SG',
  'SG',
  'TR',
  'SE',
  'ES',
  'DE',
  'US',
  'DE',
  'BE',
  'DE',
  'US',
  'ES',
  'CN',
  'IT',
  'CH',
  'CH',
  'CH',
  'CH',
  'BE',
  'IN',
  'SA',
  'SA',
  'US',
  'US',
  'US',
  'DE',
  'DE',
  'US',
  'AU',
  'AU',
  'CN',
  'IT',
  'CN',
  'GB',
  'BE',
  'JP',
  'US',
  'US',
  'GB',
  'DE',
  'ES'],
 ['US',
  'IL',
  'US',
  'US',
  'US',
  'US',
  'US',
  'US',
 

In [17]:
# pubyears, institutions, countries = extract_paper_details(works_IISER)

# fractions, team_sizes = calculate_austrian_collaboration_fraction_and_team_size(institutions, countries)
# fractions=np.array(fractions)
# fractions, team_sizes, stats = calculate_collaboration_stats(pubyears, institutions, countries, country_code)
# stats

In [20]:
pubyears, institutions, countries = extract_paper_details(works_IISER)


In [23]:
# institutions